In [1]:
sc

In [2]:
SAT_FN = '/Users/tashaygreen/Downloads/SAT_Results.csv'
HSD_FN = '/Users/tashaygreen/Downloads/DOE_High_School_Directory_2014-2015.csv'

In [3]:
df1 = spark.read \
            .format('com.databricks.spark.csv')\
            .option('header','true')\
            .option('inferSchema','true')\
            .option('parserLib','UNIVOCITY')\
            .load(HSD_FN)

# Task 1

You are asked to compute the average SAT Math score of all high schools with 500 students or more, for
each borough of the city. Meaning: what is the average SAT Math score of all high schools with 500 students
or more in Manhattan, in Brooklyn, in Queens, in Bronx and in Staten Island.

You must use Apache Spark for this lab. Both data sets must be loaded into RDDs, where all your
manipulations must be applied on, though you are free to transform these RDDs into Spark’s DataFrame or
SQL Context (though we have not covered this yet). The final result is expected to be a list of tuples borough
names as the first elements, and the average scores as the second.

Note 1: since the SAT Results also provide the number of test takers along with the average scores, you should
use this information in computing the exact average scores above.
Note 2: if a DBN in the SAT Results data set is not found in the High School Directory, you can safely ignore
the test scores for that school.

In [4]:
sat = sc.textFile(SAT_FN, use_unicode=False).cache()
list(enumerate(sat.first().split(',')))

[(0, 'DBN'),
 (1, 'SCHOOL NAME'),
 (2, 'Num of SAT Test Takers'),
 (3, 'SAT Critical Reading Avg. Score'),
 (4, 'SAT Math Avg. Score'),
 (5, 'SAT Writing Avg. Score')]

In [5]:
def extractScores(partId, list_of_records):
    import csv
    if partId==0:
        list_of_records.next()
    reader = csv.reader(list_of_records)
    for row in reader:
        if row[2]!='s':
            (dbn, takers, score) = (row[0], int(row[2]), int(row[4]))
            yield (dbn, (score*takers, takers))

satScores = sat.mapPartitionsWithIndex(extractScores)
satScores.take(4)

[('02M047', (6400, 16)),
 ('21K410', (207575, 475)),
 ('30Q301', (43120, 98)),
 ('17K382', (22066, 59))]

In [6]:
schools = sc.textFile(HSD_FN, use_unicode=False).cache()

In [7]:
def extractSchools(partId, list_of_records):
    import csv
    if partId==0:
        list_of_records.next()
    reader = csv.reader(list_of_records)
    for row in reader:
        if len(row)==58 and row[17].isdigit():
            (dbn, boro, total_students) = (row[0], row[2], int(row[17]))
            if total_students>500:
                yield (dbn, boro)
                
largeSchools = schools.mapPartitionsWithIndex(extractSchools)
largeSchools.take(4)

[('01M450', 'Manhattan'),
 ('01M539', 'Manhattan'),
 ('01M696', 'Manhattan'),
 ('02M374', 'Manhattan')]

In [8]:
scores = largeSchools.join(satScores).values()

In [9]:
scores.take(4)

[('Bronx', (23069, 59)),
 ('Staten Island', (52216, 107)),
 ('Bronx', (16317, 49)),
 ('Brooklyn', (33235, 85))]

In [10]:
scores.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) \
    .mapValues(lambda x: x[0]/x[1]) \
    .collect()

[('Bronx', 470),
 ('Manhattan', 514),
 ('Brooklyn', 487),
 ('Staten Island', 477),
 ('Queens', 474)]

# Task 2

We would like to know how the Math scores vary across bus lines or subway lines serving the schools. Your
task is to compute the average Math scores of all schools along each bus line and subway line. You can find the
bus and subway lines serving each school in the High School Dictionary as bus and subway columns.

The expected results are two lists:
1. A list of key/value pairs: with bus line as keys, and the average Math scores as values.
2. A list of key/value pairs: with subway line as keys, and the average Math scores as values

In [11]:
def extractSubway(partId, list_of_records):
    import csv
    import operator
    if partId==0:
        list_of_records.next()
    reader = csv.reader(list_of_records)
    for row in reader:
        if len(row)==58 and row[17].isdigit():
            (dbn, subway) = (row[0],row[11])
            #bus = bus.split(', ')
            subway = reduce(operator.add, map(lambda x: x.split(' to ')[0].split(', '), subway.split('; ')))
            yield (dbn, subway)
            
subway_avg = schools.mapPartitionsWithIndex(extractSubway)
subway_avg.take(2)

[('01M292', ['B', 'D', 'F', 'J', 'M', 'Z']), ('01M448', ['F', 'J', 'M', 'Z'])]

In [12]:
def extractBus(partId, list_of_records):
    import csv
    import operator
    if partId==0:
        list_of_records.next()
    reader = csv.reader(list_of_records)
    for row in reader:
        if len(row)==58 and row[17].isdigit():
            (dbn, bus) = (row[0],row[10])
            bus = bus.split(', ')
            #subway = reduce(operator.add, map(lambda x: x.split(' to ')[0].split(', '), subway.split('; ')))
            yield (dbn, bus)
            
bus_avg = schools.mapPartitionsWithIndex(extractBus)
bus_avg.take(2)

[('01M292', ['B39', 'M14A', 'M14D', 'M15', 'M15-SBS', 'M21', 'M22', 'M9']),
 ('01M448', ['M14A', 'M14D', 'M15', 'M21', 'M22', 'M9'])]

In [13]:
import itertools
bus_avg.join(satScores) \
          .flatMap(lambda x: itertools.product(x[1][0], [x[1][1]])) \
          .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \
          .mapValues(lambda x: x[0]/x[1]) \
          .sortBy(lambda x: -x[1]) \
          .take(50)

[('S1115', 612),
 ('M79', 594),
 ('Q42', 582),
 ('M22', 574),
 ('Bx3', 571),
 ('B52', 560),
 ('B63', 557),
 ('B69', 548),
 ('B54', 543),
 ('B25', 541),
 ('M20', 540),
 ('M9', 539),
 ('M86', 538),
 ('B65', 538),
 ('B45', 534),
 ('Bx10', 534),
 ('Bx26', 533),
 ('B103', 531),
 ('Q64', 529),
 ('Bx22', 525),
 ('M72', 523),
 ('M5', 520),
 ('B41', 520),
 ('B38', 520),
 ('Q35', 519),
 ('M66', 518),
 ('B62', 513),
 ('Q88', 508),
 ('Q84', 507),
 ('Q30', 507),
 ('Q20A', 505),
 ('S79-SBS', 505),
 ('Q31', 504),
 ('B11', 503),
 ('M35', 496),
 ('M10', 495),
 ('Q17', 495),
 ('Q28', 492),
 ('Q13', 492),
 ('S57', 490),
 ('M31', 490),
 ('Bx28', 489),
 ('B9', 489),
 ('Q76', 488),
 ('S78', 486),
 ('S74', 486),
 ('S55', 486),
 ('S76', 486),
 ('B8', 485),
 ('M21', 485)]

In [14]:
subway_avg.join(satScores) \
          .flatMap(lambda x: itertools.product(x[1][0], [x[1][1]])) \
          .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \
          .mapValues(lambda x: x[0]/x[1]) \
          .sortBy(lambda x: -x[1]) \
          .take(50)

[('3', 513),
 ('A', 510),
 ('C', 510),
 ('R', 508),
 ('G', 503),
 ('D', 502),
 ('E', 501),
 ('1', 499),
 ('SIR', 498),
 ('4', 495),
 ('N', 493),
 ('B', 491),
 ('2', 488),
 ('Q', 482),
 ('N/A', 476),
 ('5', 461),
 ('7', 457),
 ('M', 454),
 ('F', 445),
 ('J', 439),
 ('Z', 438),
 ('6', 432),
 ('S', 427),
 ('L', 426)]